In [1]:
import pandas as pd
import numpy as np
import urllib
import tqdm
import torch
import gc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gi_data = pd.read_csv("/content/drive/MyDrive/GI_Project/gi_data.csv", delimiter="\t")
gi_data

,Food Name,GI,Food Manufacturer,Product Category,Country of food production,Notes,Average carbohydrate portion (g),GL (based on average carbohydrate potion),Reference:,Subjects type,Time,Subjects Number,Year of test
0,"Cake, NS, decreased GI variant, sugar-to-flour...",20,"Bakery School, Herk-de-Stad, Belgium",Bakery Products,Belgium,NaN,30.0,6.0,"Vrolix, R. and R.P. Mensink, Variability of th...",Normal,2h,10,2010
1,"Carrot cake, prepared with wheat flour and coc...",37,NaN,Bakery Products,Philippines,NaN,30.0,11.0,"Trinidad, T.P., et al., Glycaemic index of dif...",Normal,2h,10,2002
2,Chocolate cake made from packet mix with choco...,38,"Betty Crocker, General Mills Inc., Minneapolis...",Bakery Products,USA,NaN,30.0,11.0,Sydney University's Glycemic Index Research Se...,Normal,2h,10,2000
3,Chocolate mudcake,43,NaN,Bakery Products,Australia,NaN,30.0,13.0,Sydney University's Glycemic Index Research Se...,Normal,2h,9,2009
4,Christmas fruit cake,53,NaN,Bakery Products,Australia,NaN,30.0,16.0,Sydney University's Glycemic Index Research Se...,Normal,2h,10,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4227,"Wholemeal bread (85 g), consumed with boiled e...",55,NaN,Regional or Traditional Foods,Singapore,NaN,35.0,19.0,"Shafaeizadeh, S., et al., Macronutrient Compos...",Normal,2h,14,2017
4228,ZANHUO/ManaGI Egg Yolk Pastry,28,"Hangzhou Yuzhong Health Technology Co., LTD.",Bakery Products,China,NaN,30.0,8.0,Beijing Key Laboratory of the Innovative Devel...,Normal,2h,12,2022
4229,ZANHUO/ManaGI Matcha Crispbread,53,"Hangzhou Yuzhong Health Technology Co., LTD",Crackers,China,NaN,15.0,8.0,Beijing Key Laboratory of the Innovative Devel...,Normal,2h,14,2022
4230,ZANHUO/ManaGI Oat Bread (Americano Flavor),42,"Hangzhou Yuzhong Health Technology Co., LTD",Crackers,China,NaN,15.0,6.0,Beijing Key Laboratory of the Innovative Devel...,Normal,2h,12,2022


In [ ]:
!wget 'https://fdc.nal.usda.gov/fdc-datasets/FoodData_Central_branded_food_csv_2023-04-20.zip'

--2023-11-26 20:34:39--  https://fdc.nal.usda.gov/fdc-datasets/FoodData_Central_branded_food_csv_2023-04-20.zip
Resolving fdc.nal.usda.gov (fdc.nal.usda.gov)... 52.245.234.40
Connecting to fdc.nal.usda.gov (fdc.nal.usda.gov)|52.245.234.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394760761 (376M) [application/zip]
Saving to: ‘FoodData_Central_branded_food_csv_2023-04-20.zip’

FoodData_Central_br 100%[===================>] 376.47M  41.7MB/s    in 8.0s    

2023-11-26 20:34:47 (46.9 MB/s) - ‘FoodData_Central_branded_food_csv_2023-04-20.zip’ saved [394760761/394760761]



In [ ]:
%%capture
!unzip /content/FoodData_Central_branded_food_csv_2023-04-20.zip

In [ ]:
foodDf = pd.read_csv("/content/branded apr 2023/food.csv")
brandedFoodDf = pd.read_csv("/content/branded apr 2023/branded_food.csv")
foodNutrientDf = pd.read_csv("/content/branded apr 2023/food_nutrient.csv")
nutrientDf = pd.read_csv("/content/branded apr 2023/nutrient.csv")
foodDf['publication_date'] = pd.to_datetime(foodDf['publication_date'])

In [ ]:
foodIds = dict(zip(foodDf['fdc_id'], foodDf['description']))
nutrientIds = dict(zip(nutrientDf['id'], nutrientDf['name'] + " [" + nutrientDf['unit_name'] + "]"))

In [ ]:
# Merge food and nutrition data frames, group by number of food ID's. Each food has multiple entries in the database,
# and some have more nutrition info than others. We want to take the food entries with the most data. Ties will
# be broken by the entry with the most recent publication date
foodDf = foodDf.merge(foodNutrientDf.groupby('fdc_id').count(), on='fdc_id')
foodDf.drop(foodDf.columns.difference(['fdc_id','description', 'publication_date', 'nutrient_id']), 1, inplace=True)
foodDf = foodDf.sort_values(by=['nutrient_id', 'publication_date']).groupby('description').tail(1)


<ipython-input-7-3b2f8ecaf142>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  foodDf.drop(foodDf.columns.difference(['fdc_id','description', 'publication_date', 'nutrient_id']), 1, inplace=True)


In [ ]:
foodDf

,fdc_id,description,publication_date,nutrient_id
264583,578248,HIMALAYAN GOURMET PINK SALT CRYSTALS,2019-04-01,1
365366,762152,"""BACN,SLAB,FL,11/13 DRIN,2PC,Z""",2020-02-27,1
365374,762168,"""BACN,HRI,SM,FL,14/18,10#,Z""",2020-02-27,1
373997,792369,RODENBACH Caractre rouge 20l One way keg,2020-04-01,1
373999,792373,STEENBRUGGE Double 20l One way keg,2020-04-01,1
...,...,...,...,...
1701822,2512297,Furmanos; 6/116OZ Pouch Baked Beans,2023-03-23,49
1701510,2511985,Bella Vista; 6/#10 Vegetarian Beans - Fancy,2023-03-23,50
1701544,2512019,Furmanos; 1/15OZ Spaghetti Sauce - Low Sodium,2023-03-23,51
1701562,2512037,Furmanos; 24/15OZ Spaghetti Sauce - Low Sodium,2023-03-23,51


In [ ]:
# Merge in nutrient data again, this time with our cleaned food data
foodNutrientMerged = foodNutrientDf.merge(foodDf, on='fdc_id')

In [ ]:
foodNutrientPt = pd.pivot_table(foodNutrientMerged, values='amount', index=['fdc_id'],
                       columns=['nutrient_id_x'], aggfunc='first')#lambda x: x if len(set(x)) == 1 else print(x))
foodNutrientPt

nutrient_id_x,1003,1004,1005,1007,1008,1009,1011,1012,1013,1018,...,1262,1263,1269,1292,1293,1316,1368,1403,1404,2000
fdc_id,,,,,,,,,,,,,,,,,,,,,
344604,0.81,0.41,4.07,NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,2.44
344605,0.81,0.41,4.07,NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,2.44
344646,5.20,3.70,87.80,0.8,370.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.6,1.0,NaN,NaN,NaN,NaN,35.90
344737,7.14,7.14,71.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
344922,10.80,3.00,16.90,4.4,112.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.4,1.0,NaN,NaN,NaN,NaN,1.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2512365,6.28,0.23,17.09,NaN,94.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.59
2512366,1.00,6.00,15.00,NaN,120.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
2512368,0.00,0.00,10.94,NaN,39.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.38


In [ ]:
foodNutrientPt['fdc_id'] = foodNutrientPt.index
foodNutrientPt.reset_index(drop=True, inplace=True)
foodNutrientPt = pd.merge(foodNutrientPt, foodDf[['fdc_id','description']], on='fdc_id', how='left')
foodNutrientPt

,1003,1004,1005,1007,1008,1009,1011,1012,1013,1018,...,1269,1292,1293,1316,1368,1403,1404,2000,fdc_id,description
0,0.81,0.41,4.07,NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,2.44,344604,Tutturosso Green 14.5oz. NSA Italian Diced Tom...
1,0.81,0.41,4.07,NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,2.44,344605,Tutturosso Green 14.5oz. Italian Diced Tomatoes
2,5.20,3.70,87.80,0.8,370.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.6,1.0,NaN,NaN,NaN,NaN,35.90,344646,KELLOGG APPLE JACKS .9OZ 100CT
3,7.14,7.14,71.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,344737,Carr's Crackers Pepper 4.25oz
4,10.80,3.00,16.90,4.4,112.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1.4,1.0,NaN,NaN,NaN,NaN,1.50,344922,GARDENBURGER GB VEGGIE BURGERS BLACK BEAN 3OZ ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407158,6.28,0.23,17.09,NaN,94.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.59,2512365,Bush's Organic Dark Red Kidney Beans 15 oz
407159,1.00,6.00,15.00,NaN,120.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,2512366,CAVENDISH FARMS POTATO PATTIES 6 x 5 LB
407160,0.00,0.00,10.94,NaN,39.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.38,2512368,Peterson Farms Treats Applesauce
407161,0.00,0.00,10.94,NaN,39.0,NaN,NaN,NaN,0.0,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,9.38,2512369,"Zee Zees Applesauce Cup, Cinnamon, Unsweetened..."


In [ ]:
foodNutrientPt


,1003,1004,1005,1007,1008,1009,1011,1012,1013,1018,...,1269,1292,1293,1316,1368,1403,1404,2000,fdc_id,description
0,0.81,0.41,4.07,NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,2.44,344604,Tutturosso Green 14.5oz. NSA Italian Diced Tom...
1,0.81,0.41,4.07,NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,2.44,344605,Tutturosso Green 14.5oz. Italian Diced Tomatoes
2,5.20,3.70,87.80,0.8,370.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.6,1.0,NaN,NaN,NaN,NaN,35.90,344646,KELLOGG APPLE JACKS .9OZ 100CT
3,7.14,7.14,71.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,344737,Carr's Crackers Pepper 4.25oz
4,10.80,3.00,16.90,4.4,112.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1.4,1.0,NaN,NaN,NaN,NaN,1.50,344922,GARDENBURGER GB VEGGIE BURGERS BLACK BEAN 3OZ ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407158,6.28,0.23,17.09,NaN,94.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.59,2512365,Bush's Organic Dark Red Kidney Beans 15 oz
407159,1.00,6.00,15.00,NaN,120.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,2512366,CAVENDISH FARMS POTATO PATTIES 6 x 5 LB
407160,0.00,0.00,10.94,NaN,39.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.38,2512368,Peterson Farms Treats Applesauce
407161,0.00,0.00,10.94,NaN,39.0,NaN,NaN,NaN,0.0,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,9.38,2512369,"Zee Zees Applesauce Cup, Cinnamon, Unsweetened..."


In [ ]:
foodNutrientPt.rename(columns=nutrientIds, inplace=True)
foodNutrientPt.rename(columns={'description' : 'name'}, inplace=True)
cols = list(foodNutrientPt)
cols.insert(0, cols.pop(cols.index('name')))
cols.insert(1, cols.pop(cols.index('fdc_id')))
foodNutrientPt = foodNutrientPt.loc[:, cols]
foodNutrientPt


,name,fdc_id,Protein [G],Total lipid (fat) [G],"Carbohydrate, by difference [G]",Ash [G],Energy [KCAL],Starch [G],Glucose [G],Fructose [G],...,SFA 10:0 [G],SFA 12:0 [G],PUFA 18:2 [G],"Fatty acids, total monounsaturated [G]","Fatty acids, total polyunsaturated [G]","PUFA 18:2 n-6 c,c [G]",Epigallocatechin-3-gallate [MG],Inulin [G],"PUFA 18:3 n-3 c,c,c (ALA) [G]","Sugars, total including NLEA [G]"
0,Tutturosso Green 14.5oz. NSA Italian Diced Tom...,344604,0.81,0.41,4.07,NaN,24.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,2.44
1,Tutturosso Green 14.5oz. Italian Diced Tomatoes,344605,0.81,0.41,4.07,NaN,24.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,2.44
2,KELLOGG APPLE JACKS .9OZ 100CT,344646,5.20,3.70,87.80,0.8,370.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.6,1.0,NaN,NaN,NaN,NaN,35.90
3,Carr's Crackers Pepper 4.25oz,344737,7.14,7.14,71.43,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
4,GARDENBURGER GB VEGGIE BURGERS BLACK BEAN 3OZ ...,344922,10.80,3.00,16.90,4.4,112.0,NaN,NaN,NaN,...,NaN,NaN,NaN,1.4,1.0,NaN,NaN,NaN,NaN,1.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407158,Bush's Organic Dark Red Kidney Beans 15 oz,2512365,6.28,0.23,17.09,NaN,94.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.59
407159,CAVENDISH FARMS POTATO PATTIES 6 x 5 LB,2512366,1.00,6.00,15.00,NaN,120.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
407160,Peterson Farms Treats Applesauce,2512368,0.00,0.00,10.94,NaN,39.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.38
407161,"Zee Zees Applesauce Cup, Cinnamon, Unsweetened...",2512369,0.00,0.00,10.94,NaN,39.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,9.38


In [ ]:
# Free up some fucking ram
del foodDf
del brandedFoodDf
del foodNutrientDf
del nutrientDf
gc.collect()

89

In [ ]:
foodNutrientPt.to_csv("USDA_Nutrition_DataV2.csv", index=0)

,name,fdc_id,Protein [G],Total lipid (fat) [G],"Carbohydrate, by difference [G]",Ash [G],Energy [KCAL],Starch [G],Glucose [G],Fructose [G],...,SFA 10:0 [G],SFA 12:0 [G],PUFA 18:2 [G],"Fatty acids, total monounsaturated [G]","Fatty acids, total polyunsaturated [G]","PUFA 18:2 n-6 c,c [G]",Epigallocatechin-3-gallate [MG],Inulin [G],"PUFA 18:3 n-3 c,c,c (ALA) [G]","Sugars, total including NLEA [G]"
0,Tutturosso Green 14.5oz. NSA Italian Diced Tom...,344604,0.81,0.41,4.07,NaN,24.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,2.44
1,Tutturosso Green 14.5oz. Italian Diced Tomatoes,344605,0.81,0.41,4.07,NaN,24.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,2.44
2,KELLOGG APPLE JACKS .9OZ 100CT,344646,5.20,3.70,87.80,0.8,370.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.6,1.0,NaN,NaN,NaN,NaN,35.90
3,Carr's Crackers Pepper 4.25oz,344737,7.14,7.14,71.43,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
4,GARDENBURGER GB VEGGIE BURGERS BLACK BEAN 3OZ ...,344922,10.80,3.00,16.90,4.4,112.0,NaN,NaN,NaN,...,NaN,NaN,NaN,1.4,1.0,NaN,NaN,NaN,NaN,1.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407158,Bush's Organic Dark Red Kidney Beans 15 oz,2512365,6.28,0.23,17.09,NaN,94.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.59
407159,CAVENDISH FARMS POTATO PATTIES 6 x 5 LB,2512366,1.00,6.00,15.00,NaN,120.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
407160,Peterson Farms Treats Applesauce,2512368,0.00,0.00,10.94,NaN,39.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.38
407161,"Zee Zees Applesauce Cup, Cinnamon, Unsweetened...",2512369,0.00,0.00,10.94,NaN,39.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,9.38


In [ ]:
!cp "/content/USDA_Nutrition_DataV2.csv" "/content/drive/MyDrive/GI_Project/USDA_Nutrition_DataV2.csv"

In [ ]:
foodNamesUSDA = pd.read_csv("/content/drive/MyDrive/GI_Project/USDA_Nutrition_DataV2.csv")['name'].tolist()
foodNamesGi = pd.read_csv("/content/drive/MyDrive/GI_Project/gi_data.csv", delimiter="\t")['Food Name'].to_list()

torch.save(foodNamesUSDA, "/content/drive/MyDrive/GI_Project/foodNamesUSDA.pt")
torch.save(foodNamesGi, "/content/drive/MyDrive/GI_Project/foodNamesGI.pt")


In [3]:
foodNamesUSDA = torch.load("/content/drive/MyDrive/GI_Project/foodNamesUSDA.pt")
foodNamesGi = torch.load("/content/drive/MyDrive/GI_Project/foodNamesGI.pt")


In [4]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=db5a2c9bd2da84352a3f7093857d5eccb49104c3ec744f6962f1e65b100738a5
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [5]:
from sentence_transformers import SentenceTransformer, util
sentences = ["I'm happy", "I'm full of happiness"]

model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')#, device='cuda')

#Compute embedding for both lists
embedding_1= model.encode(sentences[0], convert_to_tensor=True)

embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

util.pytorch_cos_sim(embedding_1, embedding_2)
## tensor([[0.6003]])

.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.66k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

tensor([[0.7780]], device='cuda:0')

In [ ]:
print(len(foodNamesUSDA), len(foodNamesGi))

407163 4232


In [6]:
model.device

device(type='cuda', index=0)

In [7]:
embeddingsGI = model.encode(foodNamesGi, convert_to_tensor=True)

In [8]:
embeddingsUSDA = model.encode(foodNamesUSDA, convert_to_tensor=True)


In [ ]:
# healthFoods = util.semantic_search(model.encode("health food", convert_to_tensor=True), usda_tensors, top_k=10000)
# junkFoods = util.semantic_search(model.encode("junk food", convert_to_tensor=True), usda_tensors, top_k=10000)


In [ ]:
# pd.DataFrame.from_dict({x['corpus_id']:'health' for x in healthFoods[0]}, orient='index').to_csv("/content/drive/MyDrive/GI_Project/healthFoods.csv")
# pd.DataFrame.from_dict({x['corpus_id']:'junk' for x in junkFoods[0]}, orient='index').to_csv("/content/drive/MyDrive/GI_Project/junkFoods.csv")


In [9]:
results = util.semantic_search(embeddingsGI, embeddingsUSDA, top_k=3, score_function=util.dot_score)


In [ ]:
out = []
for i, r in enumerate(results):
    # out[foodNamesGi[i]] = [(foodNamesUSDA[x['corpus_id']], x['score']) for x in r]
    out.append((foodNamesGi[i], foodNamesUSDA[r[0]['corpus_id']], r[0]['score']))
pd.DataFrame(out, columns=['gi_name', 'usda_name', 'score']).to_csv("/content/drive/MyDrive/GI_Project/FoodNameMatchesV3.csv")
out

In [30]:
df1 = pd.read_csv("/content/drive/MyDrive/FoodNameMatches.csv", index_col=0)
df2 = pd.read_csv("/content/drive/MyDrive/GI_Project/FoodNameMatchesV3.csv", index_col=0)

In [14]:
usda_data = pd.read_csv("/content/drive/MyDrive/GI_Project/USDA_Nutrition_DataV2.csv")

In [50]:
usda_data.loc[usda_data.isna().sum(axis=1).idxmax()]

name                                HIMALAYAN GOURMET PINK SALT CRYSTALS
fdc_id                                                            578248
Protein [G]                                                          NaN
Total lipid (fat) [G]                                                NaN
Carbohydrate, by difference [G]                                      NaN
                                                    ...                 
PUFA 18:2 n-6 c,c [G]                                                NaN
Epigallocatechin-3-gallate [MG]                                      NaN
Inulin [G]                                                           NaN
PUFA 18:3 n-3 c,c,c (ALA) [G]                                        NaN
Sugars, total including NLEA [G]                                     NaN
Name: 34689, Length: 105, dtype: object

In [42]:
usda_data['Sodium, Na [MG]'].max

KeyError: ignored

In [36]:
final = pd.merge(df2, usda_data,how='left',left_on='usda_name',right_on='name').drop(['name'], axis=1)


In [37]:
final.to_csv(f"/content/drive/MyDrive/GI_Project/NutritionDataUsdaGiMatched.csv", index=0)